## Imports

In [17]:
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import Request, urlopen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
from geopy import geocoders
import requests
import re

## Selenium Setup

In [18]:
def openChrome():
    portz = 9995
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument(f'--remote-debugging-port={portz}')
    chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"])
    chromeOptions.add_argument("--disable-blink-features=AutomationControlled")
    chromeOptions.add_argument("--disable-blink-features")
    # chromeOptions.add_argument("--headless")

    prefs = {"profile.managed_default_content_settings.images": 2}
    chromeOptions.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(executable_path=r"C:\Users\kylew\OneDrive\Desktop\Advent\chromedriver.exe", options=chromeOptions)    
    driver.set_window_rect(0, 0, width=360, height=600)
    driver.set_page_load_timeout(66)
    return driver

C:\Users\kylew\AppData\Local\Temp\ipykernel_25772\2252884548.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\kylew\OneDrive\Desktop\Advent\chromedriver.exe", options=option)


## Scraper Code

In [19]:
driver = openChrome()

output_file_path = 'out.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    pass

with open("seach.txt") as a:
    lines = a.readlines() 
    for line in lines:
        list_scan = []
        sech = line.strip()
        list_scan.append(sech)
        driver.get(f"https://www.google.com/search?q={sech}")
        sleep(1)

        page_source  = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        rate, total_review = "", ""
        rate = soup.find("span", class_="Aq14fc")
        if rate != None:
            rate = rate.text

            total_review =  soup.find("a", class_="hqzQac")
            if total_review != None:
                total_review = total_review.text.split(" ")[0].replace(",","")
            else:
                total_review =  soup.find("span", class_="hqzQac")
                total_review = total_review.text.split(" ")[0].replace(",","")

            print(rate, total_review)
            list_scan.append(rate)
            list_scan.append(total_review)

            list_cmt = soup.find("div", attrs={"class":"WFxqwc OTFaAf"})
            if list_cmt != None:
                list_cmt = list_cmt.find_all("div", class_="RfWLue")
                for i in list_cmt:
                    rate_3cmt = i.find("span", class_="z3HNkc").get("aria-label").split("Rated ")[1].split(" ")[0]
                    print(rate_3cmt)

                    cmt = i.find("a", class_="a-no-hover-decoration").text.replace("\n","").replace('"','')
                    print(cmt)
                    list_scan.append(cmt)
                    list_scan.append(rate_3cmt)

        list_write = []
        list_write.append(list_scan)

        # print(list_write)
        rows = list(zip(*list_write))
        cols_names = ['Google Search', 'Star Rating', 'Total Reviews', 'Recent Comment 1', 'Recent Rating 1', 'Recent Comment 2', 'Recent Rating 2', 'Recent Comment 3', 'Recent Rating 3']
        df_new_rows = pd.DataFrame(dict(zip(cols_names, rows)))
        # print(df_new_rows)
        df_existing = pd.read_excel(output_file_path)
        df_combined = pd.concat([df_existing, df_new_rows], ignore_index=True)
        df_combined.to_excel(output_file_path, index=False)

Error Find


,Property Name,Address,Price,Price Per Room,Sale Type,Cap Rate,Property Type,Property Subtype,Building Class,Lot Size,Building Size,No. Rooms,No. Stories,Year Built,Parking Ratio,Corridor,Link
0,,,,,,,,,,,,,,,,,https://www.loopnet.com/Listing/220-N-2nd-St-C...
1,,,,,,,,,,,,,,,,,https://www.loopnet.com/Listing/11069-Composit...
2,,,,,,,,,,,,,,,,,https://www.loopnet.com/Listing/9902-Gulf-Fwy-...
3,,,,,,,,,,,,,,,,,https://www.loopnet.com/Listing/1003-W-Highway...
4,,,,,,,,,,,,,,,,,https://www.loopnet.com/Listing/6351-North-Fwy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Hyatt Place Fort Worth/Hurst,"1601 Hurst Town Center Dr Hurst, TX, 76054",,,Investment,,Hospitality,Hotel,B,2.49 AC,"79,746 SF",127,6,1998,"1.49/1,000 SF",Interior,https://www.loopnet.com/Listing/1601-Hurst-Tow...
151,The Inn at Chacalaca Bend,"36298 Chachalaca Bend Los Fresnos, TX, 78566",,,Investment,,Hospitality,Hotel,B,50.23 AC,"14,463 SF",9,2,1973,"0.41/1,000 SF",Exterior,https://www.loopnet.com/Listing/36298-Chachala...
152,Best Western Plus Waller Hotel,"19720 Stokes Rd Waller, TX, 77484",,,Investment,,Hospitality,Hotel,B,1.29 AC,"23,968 SF",59,3,2018,,Interior,https://www.loopnet.com/Listing/19720-Stokes-R...
153,Heaven On Earth Inn,"801 St Joseph Pky Houston, TX, 77002",,,Investment,,Hospitality,Hotel,C,0.74 AC,"477,981 SF",150,30,1973,,,https://www.loopnet.com/Listing/801-St-Joseph-...
